In [4]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk import tokenize
import nltk
from gensim.models import KeyedVectors
from string import punctuation
import unidecode
from sklearn.model_selection import train_test_split
from sklearn import metrics
from imblearn.over_sampling import SMOTE
import numpy as np

# Uploading Dataset

In [5]:
colnames=['output','text']
df = pd.read_csv('all-data.csv', names=colnames, header=None)
all_sentences = [text for text in df.text]
words = ' '.join(all_sentences)

In [6]:
stop_words = nltk.corpus.stopwords.words('english')
token_space = tokenize.WhitespaceTokenizer()
token_punct = tokenize.WordPunctTokenizer()

# Function to transform and clean list of texts

In [7]:
def transformSentence(list_of_sentences):
    
    sentences_after_stopwords = list()
    
    for sentence in list_of_sentences:
        new_sentence = list()
        words_from_sentence = token_space.tokenize(sentence)
        for word in words_from_sentence:
            if word not in stop_words:
                new_sentence.append(word)
        sentences_after_stopwords.append(" ".join(new_sentence))

    sentences_after_stopwords_puncts = list()

    for sentence in sentences_after_stopwords:
        for punct_to_change in punctuation:
            sentence = sentence.replace(punct_to_change,"")
        sentences_after_stopwords_puncts.append(sentence)

    sentences_after_stopwords_puncts_lower = list()

    for sentence in sentences_after_stopwords_puncts:
        sentence = sentence.lower()
        sentences_after_stopwords_puncts_lower.append(sentence)

    stop_words_no_accent = list()

    for word in stop_words:
        for punct_to_change in punctuation:
            word = word.replace(punct_to_change,"")
        stop_words_no_accent.append(word)
    
    sentences_after_stopwords_puncts_lower_stopwords = list()

    for sentence in sentences_after_stopwords_puncts_lower:
        new_sentence = list()
        words_from_sentence = token_space.tokenize(sentence)
        for word in words_from_sentence:
            if word not in stop_words_no_accent:
                new_sentence.append(word)
        sentences_after_stopwords_puncts_lower_stopwords.append(" ".join(new_sentence))

    sentences_after_stopwords_puncts_lower_stopwords_number = list()

    for sentence in sentences_after_stopwords_puncts_lower_stopwords:
        new_sentence = list()
        words_from_sentence = token_space.tokenize(sentence)
        for word in words_from_sentence:
            if not word.isnumeric():
                new_sentence.append(word)
            else:
                new_sentence.append("0")
        sentences_after_stopwords_puncts_lower_stopwords_number.append(" ".join(new_sentence))

    return sentences_after_stopwords_puncts_lower_stopwords_number

In [8]:
treated_sentences = transformSentence(list(df.text))

Checking the imbalance between classes

In [9]:
tfidf = TfidfVectorizer(lowercase=False,max_features=600)
vector_tfidf = tfidf.fit_transform(treated_sentences)
print(vector_tfidf.shape)
print(df.output.shape)
print(df.output.value_counts())

(4846, 600)
(4846,)
neutral     2879
positive    1363
negative     604
Name: output, dtype: int64


# Balancing the classes

In [10]:
smote = SMOTE(random_state=100)
X_resampled, Y_resampled = smote.fit_resample(vector_tfidf,df.output)
print(X_resampled.shape)
print(Y_resampled.shape)
print(Y_resampled.value_counts())

(8637, 600)
(8637,)
negative    2879
neutral     2879
positive    2879
Name: output, dtype: int64


# Spliting in Train and Test

In [11]:
test_size=0.1
X_train,X_test,Y_train,Y_test = train_test_split(X_resampled,Y_resampled,random_state = 100,test_size=test_size)

In [12]:
print("train:",X_train.shape)
print("test:",X_test.shape)

train: (7773, 600)
test: (864, 600)


# Models Testing

Logistic Regression

In [13]:
from sklearn.linear_model import LogisticRegression

In [14]:
logistic_regression = LogisticRegression(solver = 'lbfgs',penalty = 'l2',max_iter = 5000)
logistic_regression.fit(X_train,Y_train)
logistic_regression.score(X_test,Y_test)

0.7557870370370371

In [15]:
logistic_regression = LogisticRegression(solver = 'lbfgs',penalty = 'none',max_iter = 5000)
logistic_regression.fit(X_train,Y_train)
logistic_regression.score(X_test,Y_test)

0.7800925925925926

In [16]:
logistic_regression = LogisticRegression(solver = 'sag',penalty = 'l2',max_iter = 5000)
logistic_regression.fit(X_train,Y_train)
logistic_regression.score(X_test,Y_test)

0.7557870370370371

In [17]:
logistic_regression = LogisticRegression(solver = 'sag',penalty = 'none',max_iter = 5000)
logistic_regression.fit(X_train,Y_train)
logistic_regression.score(X_test,Y_test)

0.7777777777777778

Multinomial Naive Bayes

In [18]:
from sklearn.naive_bayes import MultinomialNB

In [19]:
naive_bayes = MultinomialNB(alpha=1)
naive_bayes.fit(X_train,Y_train)
naive_bayes.score(X_test,Y_test)

0.6759259259259259

Random Forest Classifier

In [20]:
from sklearn.ensemble import RandomForestClassifier

In [21]:
randomForest = RandomForestClassifier(random_state= 100)
randomForest.fit(X_train,Y_train)
randomForest.score(X_test,Y_test)

0.8587962962962963

In [22]:
from sklearn.model_selection import RandomizedSearchCV

In [23]:
# param_grid = {
#     "criterion": ['gini','entropy'],
#     "min_samples_leaf": [1, 2, 4, 8],
#     "max_features": ["auto", "sqrt", "log2"]
# }

# randomForest = RandomForestClassifier(n_estimators = 300,max_depth=None, random_state=100)
# random_cv = RandomizedSearchCV(randomForest, param_grid, n_iter=20, cv=5, n_jobs=-1, random_state = 100)
# rcv = random_cv.fit(X_train, Y_train)
# rcv.best_params_

# randomForest = RandomForestClassifier(**rcv.best_params_, random_state = 100)
# randomForest.fit(X_train,Y_train)
# randomForest.score(X_test,Y_test)

In [24]:
randomForest = RandomForestClassifier(n_estimators = 300,max_depth=None, min_samples_leaf = 1, max_features = 'log2', criterion = 'entropy', random_state = 100)
randomForest.fit(X_train,Y_train)
randomForest.score(X_test,Y_test)

0.875

Decision Tree Classifier

In [25]:
from sklearn.tree import DecisionTreeClassifier

In [26]:
decisionTree = DecisionTreeClassifier(random_state = 100)
decisionTree.fit(X_train,Y_train)
decisionTree.score(X_test,Y_test)

0.7673611111111112

In [27]:
param_grid = {
    "criterion": ['gini','entropy'],
    "splitter": ["best","random"],
    "min_samples_leaf": [1, 2, 4, 8],
    "max_features": ["auto", "sqrt", "log2"]
}

decisionTree = DecisionTreeClassifier(random_state = 100)
random_cv = RandomizedSearchCV(decisionTree, param_grid, n_iter=20, cv=5, n_jobs=-1, random_state = 100)
rcv = random_cv.fit(X_train, Y_train)
rcv.best_params_

{'splitter': 'best',
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'criterion': 'entropy'}

In [28]:
decisionTree = DecisionTreeClassifier(**rcv.best_params_,random_state = 100)
decisionTree.fit(X_train,Y_train)
decisionTree.score(X_test,Y_test)

0.7754629629629629

Support Vector Machine

In [29]:
from sklearn import svm

In [32]:
support_vector = svm.SVC(random_state = 100)
support_vector.fit(X_train,Y_train)
support_vector.score(X_test,Y_test)

0.8680555555555556

In [34]:
param_grid = {
    "kernel": ['linear','poly','rbf','sigmoid'],
    "gamma": ['scale','auto'],
    "degree": [3,5,7,9]
}

support_vector = svm.SVC(random_state = 100)
random_cv = RandomizedSearchCV(support_vector, param_grid, n_iter=20, cv=5, n_jobs=-1, random_state = 100)
rcv = random_cv.fit(X_train, Y_train)
rcv.best_params_

{'kernel': 'rbf', 'gamma': 'scale', 'degree': 9}

In [35]:

support_vector = svm.SVC(**rcv.best_params_,random_state = 100)
support_vector.fit(X_train,Y_train)
support_vector.score(X_test,Y_test)

0.8680555555555556